In [1]:
!pip install h2o

     |████████████████████████████████| 177.0 MB 15 kB/s 
  Created wheel for h2o: filename=h2o-3.36.1.1-py2.py3-none-any.whl size=177068062 sha256=e10f163824988b17e79aa3be76ba855b1106dbe08c5ea98a4a72297ffc16019c
  Stored in directory: /root/.cache/pip/wheels/a6/d9/ab/5442447c7e2ccf07f66aa8b79f3877ce5382f0b95e6c0c797b
Successfully built h2o


In [2]:
import h2o
from h2o.automl import H2OAutoML
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
import sklearn

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.15" 2022-04-19; OpenJDK Runtime Environment (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1); OpenJDK 64-Bit Server VM (build 11.0.15+10-Ubuntu-0ubuntu0.18.04.1, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp3q0h97ds
  JVM stdout: /tmp/tmp3q0h97ds/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp3q0h97ds/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,04 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.36.1.1
H2O_cluster_version_age:,20 days
H2O_cluster_name:,H2O_from_python_unknownUser_1cylpb
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [4]:
import pandas as pd
def dataSetup(name):
  X,y = sklearn.datasets.fetch_openml(data_id=name, as_frame=True, return_X_y=True)
  train = pd.concat([X, y], axis=1, join='inner')
  train.dropna()
  train = train.apply(lambda x: pd.factorize(x)[0])
  X,y = train.iloc[:,:-1], train.iloc[:, -1]
  # X = X.apply(lambda x: pd.factorize(x)[0])
  X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,y,random_state=42)
  y_train = y_train.to_frame(name="class")
  y_test = y_test.to_frame(name="class")
  test = pd.concat([X_test, y_test], axis=1, join='inner')
  train = pd.concat([X_train, y_train], axis=1, join='inner')
  test = h2o.H2OFrame(test)
  train = h2o.H2OFrame(train)
  return (test, train)

In [5]:
test, train = dataSetup(40975)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [6]:
train.shape

(1296, 7)

In [7]:
x = train.columns
y = "class"
x.remove(y)

In [8]:
train[y] = train[y].asfactor()
test[y] = test[y].asfactor()

In [9]:
aml = H2OAutoML(max_runtime_secs = 3600)
aml.train(x=x, y=y, training_frame=train)
# train = h2o.H2OFrame(train)

AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_1_AutoML_1_20220503_222705_model_108


Model Summary: 


,,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
0,,337.0,1348.0,310858.0,5.0,8.0,4.66543,1.0,25.0,13.605341




ModelMetricsMultinomial: gbm
** Reported on train data. **

MSE: 0.0006328978135281765
RMSE: 0.025157460395043386
LogLoss: 0.009018876182920068
Mean Per-Class Error: 0.0
AUC: NaN
AUCPR: NaN
Multinomial auc values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).
Multinomial auc_pr values: Table is not computed because it is disabled (model parameter 'auc_type' is set to AUTO or NONE) or due to domain size (maximum is 50 domains).

Confusion Matrix: Row labels: Actual class; Column labels: Predicted class


,0,1,2,3,Error,Rate
0,916.0,0.0,0.0,0.0,0.0,0 / 916
1,0.0,281.0,0.0,0.0,0.0,0 / 281
2,0.0,0.0,45.0,0.0,0.0,0 / 45
3,0.0,0.0,0.0,54.0,0.0,0 / 54
4,916.0,281.0,45.0,54.0,0.0,"0 / 1,296"



Top-4 Hit Ratios: 


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.993831,0.005168,0.988462,0.988417,0.996139,1.000000,0.996139
1,auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
2,err,0.006169,0.005168,0.011538,0.011583,0.003861,0.000000,0.003861
3,err_count,1.600000,1.341641,3.000000,3.000000,1.000000,0.000000,1.000000
4,logloss,0.043334,0.015424,0.060051,0.041890,0.040060,0.020197,0.054471
5,max_per_class_error,0.023251,0.030872,0.076923,0.016043,0.005747,0.000000,0.017544
6,mean_per_class_accuracy,0.992433,0.011567,0.971997,0.995989,0.998563,1.000000,0.995614
7,mean_per_class_error,0.007567,0.011567,0.028003,0.004011,0.001437,0.000000,0.004386
8,mse,0.009594,0.003979,0.012358,0.011255,0.008395,0.003170,0.012791
9,pr_auc,NaN,0.000000,NaN,NaN,NaN,NaN,NaN



Scoring History: 


,,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_classification_error,training_auc,training_pr_auc
0,,2022-05-03 23:12:43,16 min 47.187 sec,0.0,0.750000,1.386294,0.444444,NaN,NaN
1,,2022-05-03 23:12:43,16 min 47.221 sec,5.0,0.570321,0.857572,0.237654,NaN,NaN
2,,2022-05-03 23:12:43,16 min 47.258 sec,10.0,0.459833,0.614120,0.145833,NaN,NaN
3,,2022-05-03 23:12:43,16 min 47.291 sec,15.0,0.398120,0.485528,0.118827,NaN,NaN
4,,2022-05-03 23:12:43,16 min 47.325 sec,20.0,0.366888,0.421224,0.100309,NaN,NaN
5,,2022-05-03 23:12:43,16 min 47.359 sec,25.0,0.329636,0.347575,0.073302,NaN,NaN
6,,2022-05-03 23:12:43,16 min 47.394 sec,30.0,0.306542,0.304091,0.064043,NaN,NaN
7,,2022-05-03 23:12:43,16 min 47.430 sec,35.0,0.285392,0.266724,0.057870,NaN,NaN
8,,2022-05-03 23:12:43,16 min 47.465 sec,40.0,0.262060,0.230646,0.046296,NaN,NaN
9,,2022-05-03 23:12:43,16 min 47.498 sec,45.0,0.242865,0.204183,0.033179,NaN,NaN



See the whole table with table.as_data_frame()

Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,safety,485.741547,1.000000,0.333155
1,persons,400.889862,0.825315,0.274958
2,buying,253.111633,0.521083,0.173601
3,maint,183.453156,0.377676,0.125825
4,lug_boot,87.190201,0.179499,0.059801
5,doors,47.617912,0.098031,0.032660


In [ ]:
lb = aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
DeepLearning_1_AutoML_1_20220502_230216,0.545304,0.695888,0.520824,0.498911,0.501125,0.251127
StackedEnsemble_BestOfFamily_4_AutoML_1_20220502_230216,0.531832,0.694321,0.502448,0.488105,0.500474,0.250474
StackedEnsemble_BestOfFamily_3_AutoML_1_20220502_230216,0.528351,0.693936,0.506414,0.497821,0.500356,0.250356
StackedEnsemble_AllModels_2_AutoML_1_20220502_230216,0.525703,0.694536,0.505831,0.496732,0.500616,0.250616
DeepLearning_grid_3_AutoML_1_20220502_230216_model_1,0.52116,0.692616,0.516703,0.5,0.49973,0.24973
DeepLearning_grid_3_AutoML_1_20220502_230216_model_6,0.519705,0.692797,0.510532,0.497821,0.499821,0.249821
StackedEnsemble_AllModels_5_AutoML_1_20220502_230216,0.513517,0.761593,0.509218,0.496754,0.527527,0.278284
DeepLearning_grid_1_AutoML_1_20220502_230216_model_9,0.512263,0.696235,0.490404,0.5,0.501533,0.251536
StackedEnsemble_AllModels_4_AutoML_1_20220502_230216,0.508223,0.695951,0.493741,0.491394,0.501307,0.251309
XRT_1_AutoML_1_20220502_230216,0.506212,0.909044,0.4887,0.485926,0.557126,0.310389
